In [16]:
import numpy as np
from fdi_att import FDI
from pypower.api import case14
from config_mea_idx import define_mea_idx_noise
from config_se import se_config

case = case14()
idx, no_mea, noise_sigma = define_mea_idx_noise(case, choice = 'RTU')


In [23]:
case_env = FDI(case, noise_sigma, idx, fpr = 0.05)
print(f'No Measurement: {case_env.no_mea}')
result = case_env.run_opf()
z, z_noise, vang_ref, vmag_ref = case_env.construct_mea(result)
v_est, _ = case_env.ac_se_pypower(z_noise, vang_ref, vmag_ref)
residual = case_env.bdd_residual(z_noise, v_est)
print(residual)
print(case_env.bdd_threshold)

No Measurement: 122
(122,)
(122, 122)
86.24155164082396
119.87093929856714


In [29]:
import random
att_spec = {}
# Voltage angle
ang_no = np.random.randint(1, 4+1)     # Attack bus number
att_spec['ang_posi'] = random.sample(case_env.non_ref_index, ang_no)    # Attack bus position
att_spec['ang_str'] = -0.5+0.5*2*np.random.rand(ang_no)                      # Attack value
# Voltage magnitude
mag_no = np.random.randint(case_env.no_bus-1)
att_spec['mag_posi'] = random.sample(case_env.non_ref_index, mag_no)
att_spec['mag_str'] = -0.001+0.002*np.random.rand()
v_att, att_posi = case_env.gen_fdi_att(v_est=v_est, att_spec=att_spec)

In [39]:
h_est = case_env.h_x_pypower(v_est)
h_att_est = case_env.h_x_pypower(v_att)
z_noise_att = z_noise + h_att_est - h_est
v_att_est, _ = case_env.ac_se_pypower(z_noise_att, vang_ref, vmag_ref)
residual_att = case_env.bdd_residual(z_noise_att, v_att_est)
print(residual_att)
print(case_env.bdd_threshold)

86.2414788928068
119.87093929856714
